<a href="https://colab.research.google.com/github/StevenVuong/MSc_Project/blob/master/p3_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook aims to modularise some of the input functions in P3 to suit the databse**

In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pickle
import seaborn as sns

# deep learning imports
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, Convolution1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils, generic_utils, to_categorical
from keras.layers import LeakyReLU
from keras import regularizers

# to split our dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# to mount our drive
from google.colab import drive

Using TensorFlow backend.


In [2]:
# mount google drive
drive.mount('/content/gdrive')

os.chdir('/content')

# go to where the data is
print (os.listdir())
os.chdir('gdrive/My Drive/msc_project')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['.config', 'gdrive', 'sample_data']


**Load our dataframe**

In [0]:
patient_df = pd.read_pickle('processed_patient_df_TRY2.pkl')

In [4]:
# establish control and pd df's
df_control = patient_df[patient_df.Group == 0] 
df_pd = patient_df[patient_df.Group == 1] 

patient_df.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,1130198,75422,0,M,73,1,MRI,MPRAGE GRAPPA,Original,11/13/2018,DCM,5/07/2019
2,1130191,75414,0,F,73,1,MRI,Sag MPRAGE GRAPPA,Original,12/13/2018,DCM,4/24/2019
3,1125041,74375,0,F,59,1,MRI,MPRAGE_GRAPPA,Original,9/06/2018,DCM,4/24/2019
4,1003469,72138,0,F,55,1,MRI,MPRAGE GRAPPA,Original,2/19/2018,DCM,4/24/2019
5,1119693,71935,1,M,66,1,MRI,MPRAGE GRAPPA,Original,4/03/2018,DCM,4/24/2019


**Load our file**

In [0]:
# for each set of training data in our pickle
# load
# process
# train our model in this
# 100 Test, 15% of 600 Validation, 85% of 600 Training. 700 Slices in total
# Save our model, tweak and evaluate etc..

In [0]:
# gets our y-values and converts to keras, one hot encoded outputs
def get_y_values(total_slices_info):
  
  y_values = [s[2] for s in total_slices_info]

  y_values = np.array(to_categorical(y_values, 2))
  
  return y_values

In [0]:
def initialise_model():

  # compile our model
  model = Sequential()

  model.add(Convolution3D(filters=8, kernel_size=2, padding='same', input_shape=(200,200,160,1)))
            #,kernel_regularizer=regularizers.l2(0.05),use_bias = True)) # padding on first one only?
  model.add(LeakyReLU(alpha=0.01)) # set to 0.01
  model.add(MaxPooling3D(pool_size=2)) # pool_size=2
  ## model.add(BatchNormalization(momentum=0.99)) # something to consider next level, moving mean and variance

  model.add(Convolution3D(filters=16, kernel_size=2))
  model.add(LeakyReLU(alpha=0.01)) 
  model.add(MaxPooling3D(pool_size=2))
  ## model.add(BatchNormalization(momentum=0.99))

  model.add(Convolution3D(filters=32, kernel_size=3))
  model.add(LeakyReLU())
  model.add(MaxPooling3D(pool_size=2))
  ## model.add(BatchNormalization(momentum=0.99))

  model.add(Convolution3D(filters=64, kernel_size=3))
  model.add(LeakyReLU(alpha=0.01))
  model.add(MaxPooling3D(pool_size=2))
  ## model.add(BatchNormalization(momentum=0.99))

  model.add(Convolution3D(filters=128, kernel_size=2))
  model.add(LeakyReLU(alpha=0.01))
  model.add(MaxPooling3D(pool_size=2))
  ## model.add(BatchNormalization(momentum=0.99))

  model.add(Convolution3D(filters=256, kernel_size=2))
  model.add(LeakyReLU(alpha=0.01))
  model.add(MaxPooling3D(pool_size=2))
  ## model.add(BatchNormalization(momentum=0.99))

  model.add(Flatten())
  # model.add(Dropout(0.45)) # add dropout to prevent overfitting
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.01))

  # model.add(Dropout(0.2))
  model.add(Dense(2, activation='softmax'))

  model.compile(optimizer=Adam(lr=0.00005), loss='categorical_crossentropy',metrics = ['accuracy']) # metrics=['categorical_accuracy']

  # experiment with literally everything?... Random Search with optimisers
            
  return model

In [0]:
# train on all, perhaps have different conditions for when we reach our last one
total_slices_train = os.listdir('stored_batches')

# Initialise Model!
# model = initialise_model()

# Load Model (If not this, then must initialise)
model = load_model('my_model_1.h5')

# initialise what we want to plot out later
history_array = []
score_array = []

num_iterations = 2 # number of times we want to loop it
for iteration in range(num_iterations):
  
  # load and fit our model for our instances
  for tsf in total_slices_train:
    pkl_path = 'stored_batches/'+tsf
  
    # load pickle file
    with open(pkl_path, 'rb') as f:
      total_slices, total_slices_info = pickle.load(f)

    # convert to numpy array
    total_slices = np.array(total_slices)

    # process y-values
    y_values = get_y_values(total_slices_info)
    
    # split into training and test set
    X_train, X_test, y_train, y_test = train_test_split(total_slices, y_values, test_size=0.1, shuffle=True)

    # fit our model ## can play with batch size
    hist = model.fit(x=X_train, y=y_train, batch_size=1, epochs=1, verbose=1, validation_split=0.2, shuffle=True)
    history_array.append(hist)
    
    # evaluate our model
    score = model.evaluate(X_test, y_test, verbose=1)
    score_array.append(score)
    
    print ("For iteration: %d, batch %s, Loss: %f, Accuracy: %f " % (iteration, tsf[-5], score[0], score[1]))
    
    # test on the fly, need to modify to split to training and test

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# save the model to use in future
model.save('my_model_1.h5', overwrite=True)  # creates a HDF5 file 'my_model.h5'

**Need script of how well our model does. Can plot the train accuracy, test accuracy and validation accuracy at once**

In [0]:
pickle.dump( [history_array, score_array], open( "model_1_records.pkl", "wb" ) )